A copy of the database schema is below is <a href="https://s3.amazonaws.com/dq-content/191/chinook-schema.svg"  target="_blank">here</a> or <a href="http://lh4.ggpht.com/_oKo6zFhdD98/SWFPtyfHJFI/AAAAAAAAAMc/GdrlzeBNsZM/s800/ChinookDatabaseSchema1.1.png"  target="_blank">here</a>

we will need to come back to it often to consult the schema as we write the queries.

<h2>Create a helper Function</h2>
<ul>
<il>We'll create some helper functions in python to save some time
<il>We'll use a 
<a href="https://docs.python.org/3/library/sqlite3.html#using-the-connection-as-a-context-manager"  target="_blank">
context manager </a> to handle the connection to the SQLite database

In [2]:
import sqlite3
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [3]:
db = 'chinook.db'
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

In [4]:
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)

In [5]:
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [6]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


<h2>Selecting New Albums to Purchase

In [19]:
albums_to_purchase = '''
    SELECT
        g.name gener_name,
        COUNT(il.invoice_id) tracks_sold
    FROM genre g
    INNER JOIN track t ON t.genre_id = g.genre_id 
    INNER JOIN invoice_line il ON il.track_id = t.track_id
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    WHERE i.billing_country = "USA"
    GROUP BY gener_name
    ORDER BY 2 DESC
    LIMIT 10;
    '''
run_query(albums_to_purchase)

,gener_name,tracks_sold
0,Rock,561
1,Alternative & Punk,130
2,Metal,124
3,R&B/Soul,53
4,Blues,36
5,Alternative,35
6,Latin,22
7,Pop,22
8,Hip Hop/Rap,20
9,Jazz,14


In [20]:
albums_to_purchase = '''
WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
'''

run_query(albums_to_purchase)

,genre,tracks_sold,percentage_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321
